In [1]:
import sys
from pathlib import Path
import os

In [2]:
project_working_dir = str(Path(sys.path[0]).parent)
sys.path += [project_working_dir]
os.chdir(project_working_dir)

In [3]:
from src.utils.read_obj import parse_obj
from src.utils.file_io import read_json
from src.simulation.setup.extract_clothing_vertex_data import extract_all_piece_vertices
from src.parameters import AVATAR_SCALING

In [4]:
avatar_mesh = parse_obj('./assets/BodyMesh.obj', './assets/BodyAnnotations.json')
avatar_mesh.scale_vertices(AVATAR_SCALING)

clothing_data = read_json('./assets/sewing_shirt.json')
all_pieces, sewing = extract_all_piece_vertices(clothing_data, avatar_mesh)

## Profile cpu version

In [7]:
trimesh = avatar_mesh.trimesh

In [6]:
vertices = all_pieces["L-1"].mesh.vertices_3d

In [12]:
%timeit trimesh.contains(vertices)

2.19 s ± 394 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit trimesh.nearest.on_surface(vertices)

683 ms ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit all_pieces["L-1"].mesh.offset_vertices((0, 0, 0))

346 µs ± 50 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
